In [ ]:
#목돈마련 (DRA -> GBI )
import pandas as pd
from dra import dynamic_risk_allocation
from gbi import gbi_allocation
from block_definitions import classify_assets_by_blocks

data_path = '/Users/heunjjang/Desktop//sample.csv'
data = pd.read_csv(data_path)

l1_assets, l2_indices, l3_markets, l4_etfs = classify_assets_by_blocks()

# 사용자 입력
goal_period = int(input("투자기간 (년): "))
risk_preference = int(input("위험 선호도 (1: 매우 안전, 5: 매우 공격적): "))
risk_aversion = 6 - risk_preference  

final_allocations = {}
upper_weights = {"L1": 1.0}

# L1: DRA
final_allocations["L1"] = dynamic_risk_allocation(data[l1_assets], risk_aversion, goal_period)

# L2 ~ L4: GBI
for level, assets, prev_level in [
    ("L2", l2_indices, "L1"),
    ("L3", l3_markets, "L2"),
    ("L4", l4_etfs, "L3")
]:
    upper_weight = sum(final_allocations[prev_level].values())
    goals = [
        {
            "name": level,
            "assets": assets,
            "amount": upper_weight,
            "period": goal_period
        }
    ]
    allocations = gbi_allocation(goals, data, risk_aversion)
    final_allocations[level] = allocations[level]

print("\n### 목돈 마련 포트폴리오 ###")
l4_allocation_list = [f"{asset}: {weight * 100:.2f}%" for asset, weight in final_allocations["L4"].items()]
print(l4_allocation_list)



### 목돈 마련 포트폴리오 ###
['REET: 5.00%', 'USO: 5.00%', 'MTUM: 5.00%', 'SPLV: 5.00%', 'SRVR: 5.00%', 'XHB: 5.00%', 'SLV: 5.00%', 'JPST: 64.99%']
